In [2]:
import os
os.environ["OPENCV_IO_ENABLE_OPENEXR"] = "1"
import cv2

import numpy as np
from pathlib import Path

pathMain = Path(r"[main path]")
pathLocalPos3d = pathMain / "AT_LocalPos3d_Raw"
pathObjectIdx = pathMain / "AT_ObjectIdx_Raw"
pathObjectLoc3d = pathMain / "AT_ObjectLoc3d_Raw"
pathLabel = pathMain / "AT_Label/full_res/SemSeg"

sFrame1:str = "Frame_0010.exr"
sFrame2:str = "Frame_0011.exr"

def LoadImages(_sFrame:str):
    pathImgLocalPos3d = pathLocalPos3d / _sFrame
    pathImgObjectIdx = pathObjectIdx / _sFrame
    pathImgObjectLoc3d = pathObjectLoc3d / _sFrame

    imgLocalPos3d = cv2.imread(
        pathImgLocalPos3d.as_posix(),
        cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH | cv2.IMREAD_UNCHANGED,
    )

    imgObjectIdx = cv2.imread(
        pathImgObjectIdx.as_posix(),
        cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH | cv2.IMREAD_UNCHANGED,
    )

    imgObjectLoc3d = cv2.imread(
        pathImgObjectLoc3d.as_posix(),
        cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH | cv2.IMREAD_UNCHANGED,
    )

    return imgLocalPos3d, imgObjectIdx, imgObjectLoc3d
# enddef

imgPos1, imgObj1, imgLoc1 = LoadImages(sFrame1)
imgPos2, imgObj2, imgLoc2 = LoadImages(sFrame2)

aOffset = np.array([[[1e4, 1e4, 1e4]]])
imgPos1 = imgPos1 - aOffset
imgPos2 = imgPos2 - aOffset

imgLoc1 = imgLoc1 - aOffset
imgLoc2 = imgLoc2 - aOffset

iChX: int = 2
iChY: int = 1
iChZ: int = 0

# print(np.min(imgPos1))
# print(np.max(imgPos1))


In [3]:
iRows, iCols, iChan = imgObj1.shape
imgObjFlat1 = imgObj1.reshape(-1, iChan)
aU, aMaskObjIdx1 = np.unique(imgObjFlat1, axis=0, return_inverse=True)
imgMaskObjIdx1 = aMaskObjIdx1.reshape(iRows, iCols)


In [4]:
# imgObjFlat2 = imgObj2.reshape(-1, iChan)
# aU2, aMaskObjIdx2 = np.unique(imgObjFlat2, axis=0, return_inverse=True)
# imgMaskObjIdx2 = aMaskObjIdx2.reshape(iRows, iCols)


In [ ]:
import matplotlib.pyplot as plt
from scipy import spatial
from scipy import interpolate

iObjIdx = 2
# plt.imshow(imgInv)
imgMask1 = imgMaskObjIdx1 == iObjIdx
imgObjPos1 = np.zeros((iRows, iCols, 4), dtype=float)
imgObjPos1[imgMask1, 0:3] = imgPos1[imgMask1, :]
imgObjPos1[imgMask1, 3] = 1.0
plt.imshow(imgObjPos1)


In [ ]:

imgMask2 = np.all(imgObj2 == aU[iObjIdx][np.newaxis, np.newaxis, :], axis=2)
imgObjPos2 = np.zeros((iRows, iCols, 4), dtype=float)
imgObjPos2[imgMask2, 0:3] = imgPos2[imgMask2, :]
imgObjPos2[imgMask2, 3] = 1.0
plt.imshow(imgObjPos2)


In [7]:
aIdxFlat1 = np.argwhere(imgMask1)
aPosFlat1 = imgPos1[imgMask1, :].reshape(-1, 3)
aPosFlat2 = imgPos2[imgMask2, :].reshape(-1, 3)



In [8]:

xTri2 = spatial.Delaunay(aPosFlat2)

In [9]:
aMeshPixX, aMeshPixY = np.meshgrid(np.arange(0, iCols), np.arange(0, iRows))

# Flatten mesh pixel coordinates and select only those of valid pixels
aMask2 = imgMask2.flatten()
aPixX = np.ndarray.flatten(aMeshPixX)[aMask2]
aPixY = np.ndarray.flatten(aMeshPixY)[aMask2]

# Calculate interpolators
ipolX = interpolate.LinearNDInterpolator(xTri2, aPixX)
ipolY = interpolate.LinearNDInterpolator(xTri2, aPixY)

# Apply interpolation for norm colors of A to obtain mapped
# pixel coordinates for image A in image B.
aPixMapX = ipolX(aPosFlat1)
aPixMapY = ipolY(aPosFlat1)


In [10]:

# aPixRC = np.stack((aPixY, aPixX), axis=1)
aPixMapXY = np.stack((aPixMapX, aPixMapY), axis=1)

aPixMapMask = np.all(np.logical_not(np.isnan(aPixMapXY)), axis=1)

aPixXY = aIdxFlat1[aPixMapMask][:,::-1]
aPixMapXY = aPixMapXY[aPixMapMask]

aFlowLines = np.stack((aPixXY, aPixMapXY), axis=1)



In [26]:
# Write flow image
aFlowXY = aPixMapXY - aPixXY

aIdxFlatMap1 = aIdxFlat1[aPixMapMask]
aIdxPos = aIdxFlatMap1[:,0] * iCols + aIdxFlatMap1[:,1]

imgValidMap = np.zeros((iRows, iCols), dtype=bool)
aValidMapFlat = imgValidMap.flatten()
aValidMapFlat[aIdxPos] = True
imgValidMap = aValidMapFlat.reshape(iRows, iCols)

# imgFlowMask1 = imgMask1.flatten()[aIdxFlatMap1].reshape(iRows, iCols)

imgFlow = np.zeros((iRows, iCols, 4), dtype=np.float32)
imgFlow[imgValidMap, iChX] = aFlowXY[:,0]
imgFlow[imgValidMap, iChY] = aFlowXY[:,1]
imgFlow[imgValidMap, iChZ] = iObjIdx
imgFlow[imgValidMap, 3] = 1.0

pathFlow = pathMain / "Flow.exr"
cv2.imwrite(pathFlow.as_posix(), imgFlow)


# plt.imshow(imgValidMap)

True

In [ ]:
# Plot flow image
from matplotlib.collections import LineCollection

clnFlow = LineCollection(aFlowLines[::500], linewidths=1.0, cmap="jet", alpha=0.5)

figFlow, axFlow = plt.subplots()
axFlow.imshow(imgObjPos1, alpha=0.5)
axFlow.imshow(imgObjPos2, alpha=0.5)

axFlow.add_collection(clnFlow)
axFlow.set_xlim(0, iCols)
axFlow.set_ylim(iRows, 0)
axFlow.margins(0.1)


In [ ]:

# imgObjPos2 = np.zeros((iRows, iCols, 4), dtype=float)
# imgObjPos2[imgMask1, 0:3] = imgPos1[imgMask1, :]
# imgObjPos2[imgMask1, 3] = 1.0
# plt.imshow(imgObjPos1)

# aMask = aMaskObjIdx == iObjIdx

# aPos1 = imgPos1.reshape(-1, 3)[aMask]
# aPos2 = imgPos2.reshape(-1, 3)[aMask]

# aCol = np.linspace(0, iCols - 1, iCols)
# aRow = np.linspace(0, iRows - 1, iRows)
# aMeshPixX, aMeshPixY = np.meshgrid(aX, aY)


# aMeshPixX, aMeshPixY = np.meshgrid(np.arange(0, iCols), np.arange(0, iRows))

# # Flatten mesh pixel coordinates and select only those of valid pixels
# aPixX = np.ndarray.flatten(aMeshPixX)[aMask]
# aPixY = np.ndarray.flatten(aMeshPixY)[aMask]


In [ ]:
imgDx = imgObjPos1
plt.imshow(imgDx)

In [8]:
aCol = np.linspace(0, iCols - 1, iCols)
aRow = np.linspace(0, iRows - 1, iRows)

ipolObjPos1 = interpolate.RegularGridInterpolator((aRow, aCol), imgObjPos1)


In [ ]:
aVals = 

In [9]:

# Triangulate image data of B
# Takes ages!
# xTri2 = spatial.Delaunay(aPos2) #, qhull_options="QJ Qbb")


In [ ]:

# Calculate interpolators
xInterX = interpolate.LinearNDInterpolator(xTri2, aPixX)
xInterY = interpolate.LinearNDInterpolator(xTri2, aPixY)

# Apply interpolation for norm colors of A to obtain mapped
# pixel coordinates for image A in image B.
aPixMapX = xInterX(aPos1)
aPixMapY = xInterY(aPos1)

# Calculate difference between mapped and original pixel positions
aPixDiffX = aPixX - aPixMapX
aPixDiffY = aPixY - aPixMapY
